In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import networkx as nx
import sys
import os
import pandas as pd
import spycon
import glob

#https://github.com/christiando/spycon
#requires pip install git+https://github.com/christiando/spycon
#https://github.com/pwollstadt/IDTxl
#requires pip install git+https://github.com/pwollstadt/IDTxl
import spycon_utils

from typing import Tuple

from sklearn.cluster import DBSCAN

sys.path.append("..")
from analysis_package import maxlab_analysis as mla

import shutil

Jpype is not available on this system. Install it from https://pypi.python.org/pypi/JPype1 to use JAVA/JIDT-powered CMI estimation.



  File "c:\Users\mrwun\anaconda3\envs\func-conn\Lib\site-packages\idtxl\estimators_jidt.py", line 9, in <module>
    import jpype as jp


Using cpu device


In [2]:
def get_filepath(div, well, remove_bursts = None): #In theory, can put in a parameter for all options, in order
    filepath = f"R:/Experimental Data/nathan_senior_project_analysis/stim_removal_network_graphs/DIV{div}/well_no={well}/"
    if remove_bursts is not None:
        filepath += f"remove_bursts={remove_bursts}/"
    return filepath

In [8]:
days = [30, 33, 34, 35, 36, 37, 38, 40, 41]
for div in days:
    for well_no in range(6):
        for remove_bursts in [False, True]:
            new_dir = f"data/images/remove_bursts={remove_bursts}/well_no={well_no}/"
            if not os.path.exists(new_dir):
                os.makedirs(new_dir)

            old_dir = get_filepath(div, well_no, remove_bursts)
            try:
                shutil.copy(f"{old_dir}network_preview_2.png", f"{new_dir}DIV{div}_network_preview_2.png")
            except FileNotFoundError as err:
                print(err)



[Errno 2] No such file or directory: 'R:/Experimental Data/nathan_senior_project_analysis/stim_removal_network_graphs/DIV35/well_no=0/remove_bursts=False/network_preview_2.png'
[Errno 2] No such file or directory: 'R:/Experimental Data/nathan_senior_project_analysis/stim_removal_network_graphs/DIV35/well_no=0/remove_bursts=True/network_preview_2.png'
[Errno 2] No such file or directory: 'R:/Experimental Data/nathan_senior_project_analysis/stim_removal_network_graphs/DIV35/well_no=1/remove_bursts=False/network_preview_2.png'
[Errno 2] No such file or directory: 'R:/Experimental Data/nathan_senior_project_analysis/stim_removal_network_graphs/DIV35/well_no=1/remove_bursts=True/network_preview_2.png'
[Errno 2] No such file or directory: 'R:/Experimental Data/nathan_senior_project_analysis/stim_removal_network_graphs/DIV35/well_no=2/remove_bursts=False/network_preview_2.png'
[Errno 2] No such file or directory: 'R:/Experimental Data/nathan_senior_project_analysis/stim_removal_network_graphs